In [1]:
PROJECT_ID = "nasa-genie-dev"
REGION = "us-central1"
import os
os.environ["PROJECT_ID"] = PROJECT_ID

In [2]:
import sys
sys.path.append("../../common/src")
sys.path.append("../src")

In [3]:
ONEDRIVE_CLIENT_ID = "821b598e-4662-4f63-a858-e57837256e7f"
os.environ["ONEDRIVE_CLIENT_ID"] = ONEDRIVE_CLIENT_ID
ONEDRIVE_TENANT_ID = "4b9ba20d-5ed9-4ed9-b867-ec77f339bc9c"
os.environ["ONEDRIVE_TENANT_ID"] = ONEDRIVE_TENANT_ID

In [4]:
PG_HOST=None
del os.environ["PG_HOST"]

In [5]:
import os
from typing import List
from common.utils.logging_handler import Logger
from common.models import QueryEngine
from config import (PROJECT_ID,
                    ONEDRIVE_CLIENT_ID,
                    ONEDRIVE_TENANT_ID,
                    ONEDRIVE_CLIENT_SECRET,
                    ONEDRIVE_PRINCIPLE_NAME)
from llama_index.readers.microsoft_onedrive import OneDriveReader
from services.query.data_source import DataSource, DataSourceFile
from services.query.sharepoint_datasource import SharePointDataSource
from utils.gcs_helper import create_bucket, upload_to_gcs

INFO: [config/config.py:57 - <module>()] Namespace File not found, setting job namespace as default


/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:166: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:329: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_de

ERROR: [config/model_config.py:591 - instantiate_model_class()] Cannot load langchain model class ChatCohere


/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:166: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:329: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_de

INFO: [config/config.py:107 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:108 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:109 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:110 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:111 - <module>()] ENABLE_TRUSS_LLAMA2 = True
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [matching_engine]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [127.0.0.1]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]
ERROR: [config/vector_store_config.py:77 - <module>()] Cannot connect to pgvector instance at 127.0.0.1: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)
INFO: [config/onedrive_config.py:30 - <module>()] ONEDRIVE_CLIENT_I

In [6]:
doc_url = "shpt:///NASA Demo"

In [7]:
from google.cloud import storage
storage_client = storage.Client(project=PROJECT_ID)

In [8]:
from common.models import QueryEngine
q_engine = QueryEngine.find_by_name("Vertex Search")
bucket_name = SharePointDataSource.downloads_bucket_name(q_engine)
shpt_ds = SharePointDataSource(storage_client, bucket_name=bucket_name)

In [9]:
temp_dir = "/tmp/nasa"
downloaded_docs = shpt_ds.download_documents(doc_url, temp_dir)

INFO: [utils/gcs_helper.py:47 - create_bucket()] Bucket nasa-genie-dev-downloads-vertex-search already exists.
INFO: [utils/gcs_helper.py:28 - clear_bucket()] Deleting all objects from GCS bucket nasa-genie-dev-downloads-vertex-search
INFO: [utils/gcs_helper.py:35 - clear_bucket()] 1 files deleted


In [10]:
downloaded_docs[0].__dict__

{'doc_name': '2021_alphabet_annual_report.pdf',
 'src_url': '01YUFKCQTIB3XGNELZLZBJUPVPMBPDBWZ6',
 'local_path': '/tmp/nasa/2021_alphabet_annual_report.pdf',
 'gcs_path': 'https://storage.googleapis.com/nasa-genie-dev-downloads-vertex-search/2021_alphabet_annual_report.pdf'}

In [ ]:
downloaded_docs = []

# extract folder name from url
sharepoint_folder = doc_url.split("shpt://")[1]

if shpt_ds.bucket_name is None:
  Logger.error(
  f"ERROR: Bucket name for SharepointDataSource {doc_url} not set. "
  f"Scraped files not uploaded to Google Cloud Storage")
else:
  # ensure downloads bucket exists, and clear contents
  create_bucket(shpt_ds.storage_client, shpt_ds.bucket_name)

In [ ]:
# download files to local storage
loader = OneDriveReader(
    client_id=ONEDRIVE_CLIENT_ID,
    tenant_id=ONEDRIVE_TENANT_ID,
    client_secret=ONEDRIVE_CLIENT_SECRET,
    userprincipalname=ONEDRIVE_PRINCIPLE_NAME,
    folder_path=sharepoint_folder,
    recursive=False
)

In [ ]:
access_token = loader._authenticate_with_msal()

In [ ]:
temp_dir = '/tmp/nasa'
subfolder_metadata = loader._connect_download_and_return_metadata(
    access_token, 
    temp_dir, 
    sharepoint_folder, 
    False, 
    userprincipalname=ONEDRIVE_PRINCIPLE_NAME,
    isRelativePath=True
)


In [ ]:
subfolder_metadata

In [ ]:
[a for a in subfolder_metadata]

In [ ]:
documents = loader.load_data(recursive=False)

In [ ]:
documents

In [ ]:
documents[0].metadata

In [ ]:
documents[2].text

In [ ]:
len(documents)

In [ ]:
documents[2].text